## Классификация в задаче анализа тональности

Используя данные отзывов IMDB, построим модели лог-регресии, RNN и CNN для классификации документов на позитивный и негативный классы.

Источник изложения: https://github.com/bentrevett/pytorch-sentiment-analysis

В предположении, что PyTorch уже установлен, поставим дополнительные модули и загрузим модель для токенизации:

In [2]:
#uncomment for linux or colab
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl

In [3]:
!pip3 install torchvision

    100% |████████████████████████████████| 61kB 1.7MB/s 
    100% |████████████████████████████████| 2.0MB 6.5MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [4]:
!pip3 install torchtext

    100% |████████████████████████████████| 71kB 1.4MB/s 


In [5]:
!pip3 install spacy

In [6]:
!python3.6 -m spacy download en

    100% |████████████████████████████████| 37.4MB 76.5MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



Загрузим датасет и получим из него выборку:

In [14]:
import torch
from torchtext import data
from torchtext import datasets

print(torch.__version__)

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

train_src, test = datasets.IMDB.splits(TEXT, LABEL)

0.4.1


aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 36.1MB/s]


Для начала обучим лог-регрессию на счётчиках слов:

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
train_counters = vect.fit_transform([' '.join(sample.text) for sample in train_src])
test_counters = vect.transform([' '.join(sample.text) for sample in test])

_transform_label = lambda x: 1 if x == 'pos' else 0

train_labels = [_transform_label(sample.label) for sample in train_src]
test_labels = [_transform_label(sample.label) for sample in test]

In [16]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_counters, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
from sklearn.metrics import f1_score
import numpy as np

answers = model.predict(test_counters)
print(f1_score(test_labels, answers))

0.8646446720980883


**Задание 1:** Заменить оценивание при фиксированном пороге (по-умолчанию у метода predict он равен 0.5) на оценку по сетке порогов и посмотрите, при каком пороге достигается наилучший результат (здесь, в силу простоты задачи, он будет выражен слабо, но в реальности, при работе с несбалансированными выборками, это могут быть дестяки пунктов f1-меры). 

In [30]:
for threshold in np.linspace(0.1, 0.9, 9):
  pred = [1 if prob1 > threshold else 0 for prob0,prob1 in model.predict_proba(test_counters) ]
  print("Threshold: {0}, F1-score: {1}".format(format(threshold,'.1f'), f1_score(test_labels, pred)))

Threshold: 0.1, F1-score: 0.8530251038468485
Threshold: 0.2, F1-score: 0.8640234067294347
Threshold: 0.3, F1-score: 0.8687172371261178
Threshold: 0.4, F1-score: 0.8690058017918459
Threshold: 0.5, F1-score: 0.8646446720980883
Threshold: 0.6, F1-score: 0.8596281485756689
Threshold: 0.7, F1-score: 0.8505474992601361
Threshold: 0.8, F1-score: 0.8338708271594178
Threshold: 0.9, F1-score: 0.7967875172097292


**Задание 2:** Поскольку данные являются сбалансированными, допустимо использовать и простую метрику accuracy. Подсчитайте её.

In [31]:
from sklearn.metrics import accuracy_score

print(accuracy_score(test_labels, answers))

0.86576


**Задание 3:** Как правило, TF-IDF подходит в качестве признаков лучше, чем счётчики встречаемости. Замените признаковое пространство на TF-IDF, обучите модель и сравните качество.

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
train_counters = vect.fit_transform([' '.join(sample.text) for sample in train_src])
test_counters = vect.transform([' '.join(sample.text) for sample in test])


model = LogisticRegression()
model.fit(train_counters, train_labels)
answers = model.predict(test_counters)
print(f1_score(test_labels, answers))

0.8827486785199423


Теперь попробуем обучить простую RNN на one-hot векторах слов. С учётом того, что в коллекции 100К уникальных слов, и векторы получаться достаточно громоздкие, урежем коллекцию до 25К слов, для всех прочих заведя токен unk (unknown). Кроме того, разобьём обучающий сет на обучение и валидацию для настройки параметров.

In [0]:
import random
train, valid = train_src.split(random_state=random.seed(SEED))

In [0]:
TEXT.build_vocab(train, max_size=25000)
LABEL.build_vocab(train)

Выше в словаре, помимо отсечения 25К наиболее частых слов, также появятся два специальных токена - unk и pad (padding).

Теперь создадим батчи, предварительно отсортировав тексты по длине, что минимизировать вставки pad-токенов:

In [0]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

Определим саму модель, в которой однослойная RNN получает на вход one-hot, пропущенный через один Dense-слой:

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        #x = [sent len, batch size]
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1, :, :], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Обучим созданную модель и оценим качество на валидационном сете. Сперва зададим оптимизатор и привяжем модель к GPU:

In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model = model.cuda()

Опишем функцию подсчёта accuracy, а также функции обучения и применения сети:

In [0]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.text.cuda()).squeeze(1)

        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text.cuda()).squeeze(1)

            loss = criterion(predictions.float(), batch.label.float().cuda())
            acc = binary_accuracy(predictions.float(), batch.label.float().cuda())

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Запустим и оценим качество на валидационном сете (по итерациям) и на тесте (в конце):

In [81]:
N_EPOCHS = 3

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01, Train Loss: 0.693, Train Acc: 50.50%, Val. Loss: 0.694, Val. Acc: 50.25%
Epoch: 02, Train Loss: 0.693, Train Acc: 49.20%, Val. Loss: 0.694, Val. Acc: 50.42%
Epoch: 03, Train Loss: 0.693, Train Acc: 49.76%, Val. Loss: 0.694, Val. Acc: 49.32%


In [82]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test Loss: 0.688, Test Acc: 53.10%


Как видим, качество низкое и не растёт с течением обучения. Попробуем улучшить модель:

- заменим связку "one-hot + dense" на предобученные эмбеддинги
- заменим RNN на BiLSTM
- добавим дропаут
- используем другой оптимизатор

Если модель вылетает по памяти, можно попробовать а) начать обучению с неё (без обучения предыдущей) и б) уменьшить размер батча

Начнём с подготовки данных:

In [83]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)


BATCH_SIZEBATCH_S  = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

.vector_cache/glove.6B.zip: 862MB [02:09, 6.67MB/s]                           
100%|█████████▉| 398134/400000 [00:18<00:00, 20812.08it/s]

**Задание 4:** Перепишите описанный выше класс RNN таким образом, чтобы в нём использовалась двунаправленная LSTM с параметром num_layers=n_layers и дропаут c заданной вероятностью dropout. Обратите внимание на то, что а) дропаут должен применяться как к весам входного слоя, так и к весам полносвязного на выходе; б) возвращаемое значение LSTM отличается от RNN наличием состояния (cell).

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(x))
        
        #embedded = [sent len, batch size, emb dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid. dim]
        #cell = [num layers * num directions, batch size, hid. dim]
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
                
        #hidden [batch size, hid. dim * num directions]
            
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

Скопируем загруженные ранее эмбеддинги во входной слой модели, заменив веса инициализации:

In [86]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

model.embedding.weight.data.copy_(pretrained_embeddings)

torch.Size([25002, 100])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4291, -0.1227,  0.0555,  ..., -0.1470,  0.2262, -0.1869],
        [-0.2643, -0.3550,  0.2888,  ..., -0.0370, -0.1209, -0.7605],
        [-0.1934,  0.3897,  0.7148,  ..., -0.4631, -0.2262,  0.5806]])

Возьмём в качестве оптимизатора алгоритм Adam, всё остальное как и раньше:

In [0]:
optimizer = optim.Adam(model.parameters())

In [0]:
criterion  = nn.BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
criterion = criterion.to(device)

Обучим и оценим модель определёнными ранее функциями. Наличие model.train() и model.eval() в их телах обеспечит включение дропаута при обучение и выключение при оценивании.

In [0]:
N_EPOCHS = 3

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

In [0]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

Попробуем улучшить модель ещё сильнее, одновременно сократив количество параметров. Для это воспользуемся свёрточными сетями.

Для создания свёрточного слоя воспользуемся nn.Conv2d, in_channels в нашем случае один (текст), out_channels -- это число число фильтров и размер ядер всех фильтров. Каждый фильтр будет иметь размерность [n x размерность эмбеддинга], где n - размер обрабатываемой n-граммы.

В PyTorch RNN требуют, что размерность батча шла вторым измерением, а CNN - первым, поэтому нужно её переставить. Важно, что предложения имели длину не меньше размера самого большого из используемых фильтров (здесь это не страшно, поскольку в используемых данных нет текстов, состоящих из пяти и менее слов).

In [0]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #x = [sent len, batch size]
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

**Задание 5:** Сейчас мы можем использовать только три различных фильтра, хотелось бы больше. Воспользуйтесь nn.ModuleList и перепишите класс сети для того, чтобы фильтров создавалось по количеству элементов в filter_sizes.

In [0]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

Как и раньше, создадим модель и загрузим предобученные эмбеддинги:


In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [0]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)

pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

In [0]:
BATCH_SIZE  = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), 
    repeat=False)

Используя определённые ранее функции, запустим обучение с оптимизатором Adam и оценим качество на валидации и тесте:

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.cuda()

In [0]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

In [0]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')